In [1]:
import sys
import os
#sys.path.insert(0, "/usr/src/app/app")
def import_path(fullpath):
    """ 
    Import a file with full path specification. Allows one to
    import from anywhere, something __import__ does not do. 
    """
    path, filename = os.path.split(fullpath)
    filename, ext = os.path.splitext(filename)
    sys.path.append(path)
    module = __import__(filename)
   # reload(module) # Might be out of date
    del sys.path[-1]
    return module
import_path("/usr/src/app/app")

<module 'app' from '/usr/src/app/app/__init__.py'>

In [2]:
from PIL import Image

from app.pkg.ml.try_on.preprocessing.aggregator import ClothProcessor
from app.pkg.ml.try_on.preprocessing.aggregator import HumanProcessor
from app.pkg.ml.try_on.lady_vton import LadyVtonAggregator
from app.pkg.ml.try_on.postprocessing.fix_face import FaceFixer

from app.pkg.ml.buffer_converters import BytesConverter
bc = BytesConverter()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
human_model = HumanProcessor()
cloth_model = ClothProcessor()
try_on_model = LadyVtonAggregator()

Some weights of the model checkpoint at laion/CLIP-ViT-H-14-laion2B-s32B-b79K were not used when initializing CLIPVisionModelWithProjection: ['text_model.encoder.layers.7.mlp.fc2.bias', 'text_model.encoder.layers.16.layer_norm2.weight', 'text_model.encoder.layers.10.self_attn.v_proj.bias', 'text_model.encoder.layers.23.self_attn.out_proj.bias', 'text_model.encoder.layers.4.self_attn.q_proj.weight', 'text_model.encoder.layers.15.mlp.fc2.bias', 'text_model.encoder.layers.17.mlp.fc1.weight', 'text_model.encoder.layers.8.self_attn.q_proj.bias', 'text_model.encoder.layers.22.mlp.fc2.weight', 'text_model.encoder.layers.19.self_attn.q_proj.weight', 'text_model.encoder.layers.18.self_attn.v_proj.weight', 'text_model.encoder.layers.6.self_attn.k_proj.bias', 'text_model.encoder.layers.23.self_attn.q_proj.bias', 'text_model.encoder.layers.12.mlp.fc2.weight', 'text_model.encoder.layers.22.self_attn.v_proj.weight', 'text_model.encoder.layers.5.self_attn.q_proj.bias', 'text_model.encoder.layers.4.se

initialization method [normal]
initialization method [normal]


2024-03-31 16:45:04,749 - [INFO] - app.pkg.ml.try_on.postprocessing.fix_face - (fix_face.py).__init__(13) - FaceFixer inited 


In [4]:
cloth_fp = "/usr/src/app/data/example/sweater.png"
human_fp = "/usr/src/app/data/example/timofey.png"

In [5]:

user_image = Image.open(human_fp)
cloth_image = Image.open(cloth_fp)


processed_user = human_model.process(user_image)

no_background_image = cloth_model.model_background(cloth_image)
no_background_image_bytes = bc.image_to_bytes(no_background_image)
result = {}
result["cloth"] = no_background_image_bytes   
processed_cloth = result

# Try on
processed_user.update(
{
    "category": "upper_body",
    "cloth": no_background_image_bytes,
}
)


In [6]:
try_on = try_on_model(processed_user)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|██████████| 20/20 [00:06<00:00,  3.17it/s]
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-03-31 16:45:13,812 - [INFO] - app.pkg.ml.try_on.postprocessing.fix_face - (fix_face.py).get_face_mask(37) - Found face on the image 


In [2]:
# bc.bytes_to_image(try_on)